In [ ]:
!git clone https://github.com/ryanzhumich/AESLC.git

Cloning into 'AESLC'...
remote: Enumerating objects: 17469, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 17469 (delta 1), reused 0 (delta 0), pack-reused 17461
Receiving objects: 100% (17469/17469), 7.36 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
pip show sacrebleu

In [ ]:
!pip install evaluate sacrebleu
!pip install evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=7916fe6f362348b4db4de7cf63511011d197292367f7c913899bbd696a5ce69d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip uninstall transformers accelerate
!pip install transformers[torch]

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.35.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.9 MB/s eta 0:00:00


In [ ]:
import os
from pathlib import Path

from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import EvalPrediction
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader


In [ ]:

def prep_training_data(output_file,input_path):

  count=0
  with open(output_file, 'w', encoding='utf-8') as train_sequence:
    for file_name in os.listdir(input_path):
      # if file_name.endswith(".txt"):
        file_path = os.path.join(input_path,file_name)
        with open(file_path,'r',encoding='utf-8') as reader:
          email,subject=reader.read().split("@subject")
          email = ' '.join(email.replace('\n',' ').split()).strip()
          subject = ' '.join(subject.replace('\n',' ').split()).strip()
          sequence = '<email> ' + email + '   <subject> ' + subject
          train_sequence.write(sequence + '\n')
  return


In [ ]:

def prep_validation_data(output_file,input_path):
    count=0
    with open(output_file, 'w', encoding='utf-8') as validation_sequence:
      for file_name in os.listdir(input_path):
        # if file_name.endswith(".txt"):
          file_path = os.path.join(input_path,file_name)
          with open(file_path,'r',encoding='utf-8') as reader:
            email,email_right = reader.read().split('@subject')
            subject,subject_right  = email_right.split('@ann0')
            ann0,ann0_right = subject_right.split('@ann1')
            ann1,ann2 = ann0_right.split('@ann2')

            email = ' '.join(email.replace('\n',' ').split()).strip()
            subject = ' '.join(subject.replace('\n',' ').split()).strip()
            ann0 = ' '.join(ann0.replace('\n',' ').split()).strip()
            ann1 = ' '.join(ann1.replace('\n',' ').split()).strip()
            ann2 = ' '.join(ann2.replace('\n',' ').split()).strip()
            sequence = '<email> ' + email + '   <subject> ' + subject
            validation_sequence.write(sequence + '\n')
    return

In [ ]:
prep_training_data('/content/training_sequence.txt','/content/AESLC/enron_subject_line/train')
prep_validation_data('/content/validating_sequence.txt','/content/AESLC/enron_subject_line/dev')

In [ ]:
checkpoint = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)  # also try gpt2-medium
model = GPT2LMHeadModel.from_pretrained(checkpoint)
extra_tokens = ["<email>", "<subject>"]
tokenizer.add_tokens(extra_tokens)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))


Embedding(50260, 768)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataset_path,tokenizer):
        super().__init__()

        email_subject_path = dataset_path

        self.data_list = []
        self.eos_token_id = tokenizer.eos_token_id

        with open(email_subject_path) as file:
          for line in file:
            data_str= line+"<eos>"
            self.data_list.append(data_str)

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, item):
        data = self.data_list[item]
        encoding = tokenizer(data, return_tensors="pt", truncation=True, padding=True)

        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
        }


In [ ]:
# train_dataset = TextDataset(tokenizer=tokenizer, file_path="/content/training_sequence.txt", block_size=128)
# val_dataset = TextDataset(tokenizer=tokenizer, file_path="/content/validating_sequence.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
train_dataset = CustomDataset( dataset_path="/content/training_sequence.txt",tokenizer=tokenizer)
val_dataset = CustomDataset(dataset_path="/content/training_sequence.txt",tokenizer=tokenizer)

In [ ]:
type(train_dataset)

__main__.CustomDataset

In [ ]:
model_output_path = "/content/gpt_model"

training_args = TrainingArguments(
          output_dir=model_output_path,
          overwrite_output_dir=True,
          evaluation_strategy = 'steps',
          eval_steps = 1000,
          # save_strategy = 'epoch',
          per_device_train_batch_size=4,  # Adjust as needed based on your GPU memory
          per_device_eval_batch_size=8,
          gradient_accumulation_steps=2,
          num_train_epochs=4,
          fp16=True
      )

In [ ]:
trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
          eval_dataset=val_dataset,
          # preprocess_logits_for_metrics=preprocess_logits_for_metrics,
          # compute_metrics=compute_metrics
  )
trainer.train()
trainer.save_model()

Step,Training Loss,Validation Loss
1000,3.344100,3.127193
2000,3.185800,2.965580
3000,3.065400,2.856760
4000,2.955300,2.782701
5000,2.905900,2.729061
6000,2.849400,2.694795
7000,2.831200,2.678843


In [ ]:
import shutil
shutil.make_archive('/content/gpt_model/checkpoint-7000', 'zip', '/content')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned GPT-2 model and modified tokenizer
model_path = "/content/gpt_model/checkpoint-4000"
# tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Set the device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)



GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

In [ ]:
# Use the model for text generation
prompt = """UBS have organized an important meeting  will be held tomorrow for all employees who :   1) have accepted offers or  2) intend to accept offers ( to the best of their knowledge) but who have issues that are being resolved  All employees in these categories should attend  Buses will be provided for those who do not have their own transport and will pick up from the south side of the north building - please assemble in the Java plaza area from 8.30am  to 9.30am ( last bus leaves  @ 9.30 am )  The meeting will last until approximately 1pm  and will be presented by senior managers from UBS on the firm and it's strategy.
There will be breakfast available at the Hotel from 9am to 10am and there will be sandwiches afterwards.
If traveling independently please make sure you leave sufficient time to park (it will be very busy) and get to the Grand ballroom  Employees need to bring their id's or drivers license with them to gain access to the meeting  I'm sorry for the short notice on this - please call out to anyone who should be attending who will not get this message in time    Tammie Schoppe  on behalf of UBS HR"""
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
attention_mask = torch.ones_like(input_ids)
pad_token_id = tokenizer.eos_token_id
output_ids = model.generate(input_ids, max_length=1024, num_return_sequences=1,attention_mask=attention_mask,
        pad_token_id=pad_token_id)

# Decode and print the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)


UBS have organized an important meeting  will be held tomorrow for all employees who :   1) have accepted offers or  2) intend to accept offers ( to the best of their knowledge) but who have issues that are being resolved  All employees in these categories should attend  Buses will be provided for those who do not have their own transport and will pick up from the south side of the north building - please assemble in the Java plaza area from 8.30am  to 9.30am ( last bus leaves  @ 9.30 am )  The meeting will last until approximately 1pm  and will be presented by senior managers from UBS on the firm and it's strategy.
There will be breakfast available at the Hotel from 9am to 10am and there will be sandwiches afterwards.
If traveling independently please make sure you leave sufficient time to park (it will be very busy) and get to the Grand ballroom  Employees need to bring their id's or drivers license with them to gain access to the meeting  I'm sorry for the short notice on this - ple